In [ ]:
# wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
# wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [1]:
import pandas as pd

In [5]:
df = pd.read_parquet("green_tripdata_2025-11.parquet", engine="pyarrow")

In [6]:
df.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [3]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [ ]:
question_3 = df[(df["lpep_pickup_datetime"] >= "2025-11-01") & (df["lpep_pickup_datetime"] < "2025-12-01") & (df["trip_distance"] <= 1)]

question_3.head()
len(question_3)

8007

In [11]:
question_4 = df[df["trip_distance"] < 100]
question_4.loc[question_4["trip_distance"].idxmax()]

VendorID                                   2
lpep_pickup_datetime     2025-11-14 15:36:27
lpep_dropoff_datetime    2025-11-14 18:40:48
store_and_fwd_flag                         N
RatecodeID                               4.0
PULocationID                             130
DOLocationID                             265
passenger_count                          2.0
trip_distance                          88.03
fare_amount                            610.6
extra                                    0.0
mta_tax                                  0.5
tip_amount                               0.0
tolls_amount                             0.0
ehail_fee                                NaN
improvement_surcharge                    1.0
total_amount                           612.1
payment_type                             2.0
trip_type                                1.0
congestion_surcharge                     0.0
cbd_congestion_fee                       0.0
Name: 18867, dtype: object

In [13]:
zones_df = pd.read_csv("taxi_zone_lookup.csv")
zones_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [22]:
pickup_zones = zones_df.rename(columns={"LocationID": "PULocationID"})
question_5 = df[(df["lpep_pickup_datetime"] > "2025-11-18") & (df["lpep_pickup_datetime"] < "2025-11-19")]
question_5_merged = question_5.merge(pickup_zones, on="PULocationID", how="left")
question_5_sum = question_5_merged.groupby("Zone", as_index=False)["total_amount"].sum()
question_5_sum.loc[question_5_sum["total_amount"].idxmax()]

Zone            East Harlem North
total_amount              9281.92
Name: 39, dtype: object

In [26]:
question_6 = df.merge(pickup_zones, on="PULocationID", how="left")
question_6_EHN = question_6[question_6["Zone"] == "East Harlem North"]
question_6_EHN.loc[question_6_EHN["tip_amount"].idxmax()] # DOLocationID = 263
zones_df[zones_df["LocationID"] == 263]

,LocationID,Borough,Zone,service_zone
262,263,Manhattan,Yorkville West,Yellow Zone
